In [1]:
import os
import torch
import pwcca
import random
import librosa
import opensmile
import numpy as np
import pandas as pd
import torch.nn as nn
import soundfile as sf
from scipy import signal
from sklearn.preprocessing import StandardScaler, normalize
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
w2v_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/afs/inf.ed.ac.uk/user/s20/s2057508/.local/lib/python3.6/site-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.codevectors', 'project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

In [3]:
# calculate correlation

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RAVDESS/Data'

# hct_frame = np.array([])
hct_phone = np.array([])
# hct_word = np.array([])
# hct_utter = np.array([])

n = 1
for dirname, dirs, filenames in os.walk(path):
    for filename in filenames:
        print(n)
        emotion = filename[7]
        file_name = os.path.join(dirname, filename)
        audioinput, sr = librosa.load(file_name, sr=16000)

        torch.cuda.empty_cache()

        egmaps = smile.process_signal(audioinput, sr).values
#         print(egmaps.shape)
    
        #w2v features
        input_values = processor(audioinput, sampling_rate=16000, return_tensors="pt").input_values.to(device)
        w2v_feats = w2v_model(input_values, output_hidden_states=True).hidden_states
        w2v0 = np.array(w2v_feats[0].detach()[0].cpu())
        w2v1 = np.array(w2v_feats[1].detach()[0].cpu())
        w2v2 = np.array(w2v_feats[2].detach()[0].cpu())
        w2v3 = np.array(w2v_feats[3].detach()[0].cpu())
        w2v4 = np.array(w2v_feats[4].detach()[0].cpu())
        w2v5 = np.array(w2v_feats[5].detach()[0].cpu())
        w2v6 = np.array(w2v_feats[6].detach()[0].cpu())
        w2v7 = np.array(w2v_feats[7].detach()[0].cpu())
        w2v8 = np.array(w2v_feats[8].detach()[0].cpu())
        w2v9 = np.array(w2v_feats[9].detach()[0].cpu())
        w2v10 = np.array(w2v_feats[10].detach()[0].cpu())
        w2v11 = np.array(w2v_feats[11].detach()[0].cpu())
        w2v12 = np.array(w2v_feats[12].detach()[0].cpu())
        
#         egmaps_frame = signal.resample(egmaps, len(w2v0))
        
        egmaps_phone = []
        for i in range(0,len(egmaps)-3, 3):
            egmaps_phone.append((egmaps[i]+egmaps[i+1]+egmaps[i+2])/3.0) #phoneme level   
#         egmaps_word=[]
#         for i in range(0, len(egmaps_phone)-5, 5):
#             egmaps_word.append((egmaps_phone[i]+egmaps_phone[i+1]+egmaps_phone[i+2]+egmaps_phone[i+3]+egmaps_phone[i+4])/5.0) #word level
#         egmaps_utter = np.mean(egmaps, axis=0).reshape(1, -1)
#         print(egmaps_frame.shape)
#         print(w2v0.shape)
        w2v0 = signal.resample(w2v0, len(egmaps_phone))
        w2v1 = signal.resample(w2v1, len(egmaps_phone))
        w2v2 = signal.resample(w2v2, len(egmaps_phone))
        w2v3 = signal.resample(w2v3, len(egmaps_phone))
        w2v4 = signal.resample(w2v4, len(egmaps_phone))
        w2v5 = signal.resample(w2v5, len(egmaps_phone))
        w2v6 = signal.resample(w2v6, len(egmaps_phone))
        w2v7 = signal.resample(w2v7, len(egmaps_phone))
        w2v8 = signal.resample(w2v8, len(egmaps_phone))
        w2v9 = signal.resample(w2v9, len(egmaps_phone))
        w2v10 = signal.resample(w2v10, len(egmaps_phone))
        w2v11 = signal.resample(w2v11, len(egmaps_phone))
        w2v12 = signal.resample(w2v12, len(egmaps_phone))

#for utterance level
#         w2v0 = np.mean(signal.resample(w2v0, 2), axis=0).reshape(1, -1)
#         w2v1 = np.mean(signal.resample(w2v1, 2), axis=0).reshape(1, -1)
#         w2v2 = np.mean(signal.resample(w2v2, 2), axis=0).reshape(1, -1)
#         w2v3 = np.mean(signal.resample(w2v3, 2), axis=0).reshape(1, -1)
#         w2v4 = np.mean(signal.resample(w2v4, 2), axis=0).reshape(1, -1)
#         w2v5 = np.mean(signal.resample(w2v5, 2), axis=0).reshape(1, -1)
#         w2v6 = np.mean(signal.resample(w2v6, 2), axis=0).reshape(1, -1)
#         w2v7 = np.mean(signal.resample(w2v7, 2), axis=0).reshape(1, -1)
#         w2v8 = np.mean(signal.resample(w2v8, 2), axis=0).reshape(1, -1)
#         w2v9 = np.mean(signal.resample(w2v9, 2), axis=0).reshape(1, -1)
#         w2v10 = np.mean(signal.resample(w2v10, 2), axis=0).reshape(1, -1)
#         w2v11 = np.mean(signal.resample(w2v11, 2), axis=0).reshape(1, -1)
#         w2v12 = np.mean(signal.resample(w2v12, 2), axis=0).reshape(1, -1)
        
#         print(w2v1.shape, egmaps_phone.shape)

        torch.cuda.empty_cache()
        
        if len(hct_phone) > 0:
            w2v0_feats = np.concatenate((w2v0_feats, w2v0), axis=0)
            w2v1_feats = np.concatenate((w2v1_feats, w2v1), axis=0)
            w2v2_feats = np.concatenate((w2v2_feats, w2v2), axis=0)
            w2v3_feats = np.concatenate((w2v3_feats, w2v3), axis=0)
            w2v4_feats = np.concatenate((w2v4_feats, w2v4), axis=0)
            w2v5_feats = np.concatenate((w2v5_feats, w2v5), axis=0)
            w2v6_feats = np.concatenate((w2v6_feats, w2v6), axis=0)
            w2v7_feats = np.concatenate((w2v7_feats, w2v7), axis=0)
            w2v8_feats = np.concatenate((w2v8_feats, w2v8), axis=0)
            w2v9_feats = np.concatenate((w2v9_feats, w2v9), axis=0)
            w2v10_feats = np.concatenate((w2v10_feats, w2v10), axis=0)
            w2v11_feats = np.concatenate((w2v11_feats, w2v11), axis=0)
            w2v12_feats = np.concatenate((w2v12_feats, w2v12), axis=0)
            hct_phone = np.concatenate((hct_phone, egmaps_phone), axis=0)
        else:
            w2v0_feats = w2v0
            w2v1_feats = w2v1
            w2v2_feats = w2v2
            w2v3_feats = w2v3
            w2v4_feats = w2v4
            w2v5_feats = w2v5
            w2v6_feats = w2v6
            w2v7_feats = w2v7
            w2v8_feats = w2v8
            w2v9_feats = w2v9
            w2v10_feats = w2v10 
            w2v11_feats = w2v11
            w2v12_feats = w2v12
            hct_phone = egmaps_phone
            
        n += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
print(hct_frame.shape, w2v1_feats.shape)

(265223, 25) (265223, 768)


In [4]:
corr_frame0, _, coefs0 = pwcca.compute_pwcca(hct_phone.T, w2v0_feats.T, epsilon=1e-10)
corr_frame1, _, coefs1 = pwcca.compute_pwcca(hct_phone.T, w2v1_feats.T, epsilon=1e-10)
corr_frame2, _, coefs2 = pwcca.compute_pwcca(hct_phone.T, w2v2_feats.T, epsilon=1e-10)
corr_frame3, _, coefs3 = pwcca.compute_pwcca(hct_phone.T, w2v3_feats.T, epsilon=1e-10)
corr_frame4, _, coefs4 = pwcca.compute_pwcca(hct_phone.T, w2v4_feats.T, epsilon=1e-10)
corr_frame5, _, coefs5 = pwcca.compute_pwcca(hct_phone.T, w2v5_feats.T, epsilon=1e-10)
corr_frame6, _, coefs6 = pwcca.compute_pwcca(hct_phone.T, w2v6_feats.T, epsilon=1e-10)
corr_frame7, _, coefs7 = pwcca.compute_pwcca(hct_phone.T, w2v7_feats.T, epsilon=1e-10)
corr_frame8, _, coefs8 = pwcca.compute_pwcca(hct_phone.T, w2v8_feats.T, epsilon=1e-10)
corr_frame9, _, coefs9 = pwcca.compute_pwcca(hct_phone.T, w2v9_feats.T, epsilon=1e-10)
corr_frame10, _, coefs10 = pwcca.compute_pwcca(hct_phone.T, w2v10_feats.T, epsilon=1e-10)
corr_frame11, _, coefs11 = pwcca.compute_pwcca(hct_phone.T, w2v11_feats.T, epsilon=1e-10)
corr_frame12, _, coefs12 = pwcca.compute_pwcca(hct_phone.T, w2v12_feats.T, epsilon=1e-10)

print('Layer 0')
# print(corr_frame0)
print(np.mean(coefs0))

print('Layer 1')
# print(corr_frame1)
print(np.mean(coefs1))

print('Layer 2')
# print(corr_frame2)
print(np.mean(coefs2))

print('Layer 3')
# print(corr_frame3)
print(np.mean(coefs3))

print('Layer 4')
# print(corr_frame4)
print(np.mean(coefs4))

print('Layer 5')
# print(corr_frame5)
print(np.mean(coefs5))

print('Layer 6')
# print(corr_frame6)
print(np.mean(coefs6))

print('Layer 7')
# print(corr_frame7)
print(np.mean(coefs7))

print('Layer 8')
# print(corr_frame8)
print(np.mean(coefs8))

print('Layer 9')
# print(corr_frame9)
print(np.mean(coefs9))

print('Layer 10')
# print(corr_frame10)
print(np.mean(coefs10))

print('Layer 11')
# print(corr_frame11)
print(np.mean(coefs11))

print('Layer 12')
# print(corr_frame12)
print(np.mean(coefs12))

Layer 0
0.5993155183309384
Layer 1
0.6201719894075368
Layer 2
0.6161960132019728
Layer 3
0.6038882775927162
Layer 4
0.5848705176524586
Layer 5
0.5627587318250595
Layer 6
0.5464921183101883
Layer 7
0.5387733871307925
Layer 8
0.5423727421807715
Layer 9
0.5663953648628862
Layer 10
0.5934620850022515
Layer 11
0.43253221493398086
Layer 12
0.5044028700686257


In [5]:
# calculate correlation

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RAVDESS/Data'

# hct_frame = np.array([])
# hct_phone = np.array([])
hct_word = np.array([])
# hct_utter = np.array([])

n = 1
for dirname, dirs, filenames in os.walk(path):
    for filename in filenames:
        print(n)
        emotion = filename[7]
        file_name = os.path.join(dirname, filename)
        audioinput, sr = librosa.load(file_name, sr=16000)

        torch.cuda.empty_cache()

        egmaps = smile.process_signal(audioinput, sr).values
#         print(egmaps.shape)
    
        #w2v features
        input_values = processor(audioinput, sampling_rate=16000, return_tensors="pt").input_values.to(device)
        w2v_feats = w2v_model(input_values, output_hidden_states=True).hidden_states
        w2v0 = np.array(w2v_feats[0].detach()[0].cpu())
        w2v1 = np.array(w2v_feats[1].detach()[0].cpu())
        w2v2 = np.array(w2v_feats[2].detach()[0].cpu())
        w2v3 = np.array(w2v_feats[3].detach()[0].cpu())
        w2v4 = np.array(w2v_feats[4].detach()[0].cpu())
        w2v5 = np.array(w2v_feats[5].detach()[0].cpu())
        w2v6 = np.array(w2v_feats[6].detach()[0].cpu())
        w2v7 = np.array(w2v_feats[7].detach()[0].cpu())
        w2v8 = np.array(w2v_feats[8].detach()[0].cpu())
        w2v9 = np.array(w2v_feats[9].detach()[0].cpu())
        w2v10 = np.array(w2v_feats[10].detach()[0].cpu())
        w2v11 = np.array(w2v_feats[11].detach()[0].cpu())
        w2v12 = np.array(w2v_feats[12].detach()[0].cpu())
        
#         egmaps_frame = signal.resample(egmaps, len(w2v0))
        
        egmaps_phone = []
        for i in range(0,len(egmaps)-3, 3):
            egmaps_phone.append((egmaps[i]+egmaps[i+1]+egmaps[i+2])/3.0) #phoneme level   
        egmaps_word=[]
        for i in range(0, len(egmaps_phone)-5, 5):
            egmaps_word.append((egmaps_phone[i]+egmaps_phone[i+1]+egmaps_phone[i+2]+egmaps_phone[i+3]+egmaps_phone[i+4])/5.0) #word level
#         egmaps_utter = np.mean(egmaps, axis=0).reshape(1, -1)
#         print(egmaps_frame.shape)
#         print(w2v0.shape)
        w2v0 = signal.resample(w2v0, len(egmaps_word))
        w2v1 = signal.resample(w2v1, len(egmaps_word))
        w2v2 = signal.resample(w2v2, len(egmaps_word))
        w2v3 = signal.resample(w2v3, len(egmaps_word))
        w2v4 = signal.resample(w2v4, len(egmaps_word))
        w2v5 = signal.resample(w2v5, len(egmaps_word))
        w2v6 = signal.resample(w2v6, len(egmaps_word))
        w2v7 = signal.resample(w2v7, len(egmaps_word))
        w2v8 = signal.resample(w2v8, len(egmaps_word))
        w2v9 = signal.resample(w2v9, len(egmaps_word))
        w2v10 = signal.resample(w2v10, len(egmaps_word))
        w2v11 = signal.resample(w2v11, len(egmaps_word))
        w2v12 = signal.resample(w2v12, len(egmaps_word))

#for utterance level
#         w2v0 = np.mean(signal.resample(w2v0, 2), axis=0).reshape(1, -1)
#         w2v1 = np.mean(signal.resample(w2v1, 2), axis=0).reshape(1, -1)
#         w2v2 = np.mean(signal.resample(w2v2, 2), axis=0).reshape(1, -1)
#         w2v3 = np.mean(signal.resample(w2v3, 2), axis=0).reshape(1, -1)
#         w2v4 = np.mean(signal.resample(w2v4, 2), axis=0).reshape(1, -1)
#         w2v5 = np.mean(signal.resample(w2v5, 2), axis=0).reshape(1, -1)
#         w2v6 = np.mean(signal.resample(w2v6, 2), axis=0).reshape(1, -1)
#         w2v7 = np.mean(signal.resample(w2v7, 2), axis=0).reshape(1, -1)
#         w2v8 = np.mean(signal.resample(w2v8, 2), axis=0).reshape(1, -1)
#         w2v9 = np.mean(signal.resample(w2v9, 2), axis=0).reshape(1, -1)
#         w2v10 = np.mean(signal.resample(w2v10, 2), axis=0).reshape(1, -1)
#         w2v11 = np.mean(signal.resample(w2v11, 2), axis=0).reshape(1, -1)
#         w2v12 = np.mean(signal.resample(w2v12, 2), axis=0).reshape(1, -1)
        
#         print(w2v1.shape, egmaps_phone.shape)

        torch.cuda.empty_cache()
        
        if len(hct_word) > 0:
            w2v0_feats = np.concatenate((w2v0_feats, w2v0), axis=0)
            w2v1_feats = np.concatenate((w2v1_feats, w2v1), axis=0)
            w2v2_feats = np.concatenate((w2v2_feats, w2v2), axis=0)
            w2v3_feats = np.concatenate((w2v3_feats, w2v3), axis=0)
            w2v4_feats = np.concatenate((w2v4_feats, w2v4), axis=0)
            w2v5_feats = np.concatenate((w2v5_feats, w2v5), axis=0)
            w2v6_feats = np.concatenate((w2v6_feats, w2v6), axis=0)
            w2v7_feats = np.concatenate((w2v7_feats, w2v7), axis=0)
            w2v8_feats = np.concatenate((w2v8_feats, w2v8), axis=0)
            w2v9_feats = np.concatenate((w2v9_feats, w2v9), axis=0)
            w2v10_feats = np.concatenate((w2v10_feats, w2v10), axis=0)
            w2v11_feats = np.concatenate((w2v11_feats, w2v11), axis=0)
            w2v12_feats = np.concatenate((w2v12_feats, w2v12), axis=0)
            hct_word = np.concatenate((hct_word, egmaps_word), axis=0)
        else:
            w2v0_feats = w2v0
            w2v1_feats = w2v1
            w2v2_feats = w2v2
            w2v3_feats = w2v3
            w2v4_feats = w2v4
            w2v5_feats = w2v5
            w2v6_feats = w2v6
            w2v7_feats = w2v7
            w2v8_feats = w2v8
            w2v9_feats = w2v9
            w2v10_feats = w2v10 
            w2v11_feats = w2v11
            w2v12_feats = w2v12
            hct_word = egmaps_word
            
        n += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
corr_frame0, _, coefs0 = pwcca.compute_pwcca(hct_word.T, w2v0_feats.T, epsilon=1e-10)
corr_frame1, _, coefs1 = pwcca.compute_pwcca(hct_word.T, w2v1_feats.T, epsilon=1e-10)
corr_frame2, _, coefs2 = pwcca.compute_pwcca(hct_word.T, w2v2_feats.T, epsilon=1e-10)
corr_frame3, _, coefs3 = pwcca.compute_pwcca(hct_word.T, w2v3_feats.T, epsilon=1e-10)
corr_frame4, _, coefs4 = pwcca.compute_pwcca(hct_word.T, w2v4_feats.T, epsilon=1e-10)
corr_frame5, _, coefs5 = pwcca.compute_pwcca(hct_word.T, w2v5_feats.T, epsilon=1e-10)
corr_frame6, _, coefs6 = pwcca.compute_pwcca(hct_word.T, w2v6_feats.T, epsilon=1e-10)
corr_frame7, _, coefs7 = pwcca.compute_pwcca(hct_word.T, w2v7_feats.T, epsilon=1e-10)
corr_frame8, _, coefs8 = pwcca.compute_pwcca(hct_word.T, w2v8_feats.T, epsilon=1e-10)
corr_frame9, _, coefs9 = pwcca.compute_pwcca(hct_word.T, w2v9_feats.T, epsilon=1e-10)
corr_frame10, _, coefs10 = pwcca.compute_pwcca(hct_word.T, w2v10_feats.T, epsilon=1e-10)
corr_frame11, _, coefs11 = pwcca.compute_pwcca(hct_word.T, w2v11_feats.T, epsilon=1e-10)
corr_frame12, _, coefs12 = pwcca.compute_pwcca(hct_word.T, w2v12_feats.T, epsilon=1e-10)

print('Layer 0')
# print(corr_frame0)
print(np.mean(coefs0))

print('Layer 1')
# print(corr_frame1)
print(np.mean(coefs1))

print('Layer 2')
# print(corr_frame2)
print(np.mean(coefs2))

print('Layer 3')
# print(corr_frame3)
print(np.mean(coefs3))

print('Layer 4')
# print(corr_frame4)
print(np.mean(coefs4))

print('Layer 5')
# print(corr_frame5)
print(np.mean(coefs5))

print('Layer 6')
# print(corr_frame6)
print(np.mean(coefs6))

print('Layer 7')
# print(corr_frame7)
print(np.mean(coefs7))

print('Layer 8')
# print(corr_frame8)
print(np.mean(coefs8))

print('Layer 9')
# print(corr_frame9)
print(np.mean(coefs9))

print('Layer 10')
# print(corr_frame10)
print(np.mean(coefs10))

print('Layer 11')
# print(corr_frame11)
print(np.mean(coefs11))

print('Layer 12')
# print(corr_frame12)
print(np.mean(coefs12))

Layer 0
0.652253153818508
Layer 1
0.6730757788810677
Layer 2
0.66855332157082
Layer 3
0.6562535025133178
Layer 4
0.637576584272603
Layer 5
0.6166318379336466
Layer 6
0.6000018620340711
Layer 7
0.5919748412321137
Layer 8
0.5939367442905545
Layer 9
0.6171276062982549
Layer 10
0.6412929872558709
Layer 11
0.5306894740328171
Layer 12
0.577477331490143


In [7]:
# calculate correlation

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RAVDESS/Data'

# hct_frame = np.array([])
# hct_phone = np.array([])
# hct_word = np.array([])
hct_utter = np.array([])

n = 1
for dirname, dirs, filenames in os.walk(path):
    for filename in filenames:
        print(n)
        emotion = filename[7]
        file_name = os.path.join(dirname, filename)
        audioinput, sr = librosa.load(file_name, sr=16000)

        torch.cuda.empty_cache()

        egmaps = smile.process_signal(audioinput, sr).values
#         print(egmaps.shape)
    
        #w2v features
        input_values = processor(audioinput, sampling_rate=16000, return_tensors="pt").input_values.to(device)
        w2v_feats = w2v_model(input_values, output_hidden_states=True).hidden_states
        w2v0 = np.array(w2v_feats[0].detach()[0].cpu())
        w2v1 = np.array(w2v_feats[1].detach()[0].cpu())
        w2v2 = np.array(w2v_feats[2].detach()[0].cpu())
        w2v3 = np.array(w2v_feats[3].detach()[0].cpu())
        w2v4 = np.array(w2v_feats[4].detach()[0].cpu())
        w2v5 = np.array(w2v_feats[5].detach()[0].cpu())
        w2v6 = np.array(w2v_feats[6].detach()[0].cpu())
        w2v7 = np.array(w2v_feats[7].detach()[0].cpu())
        w2v8 = np.array(w2v_feats[8].detach()[0].cpu())
        w2v9 = np.array(w2v_feats[9].detach()[0].cpu())
        w2v10 = np.array(w2v_feats[10].detach()[0].cpu())
        w2v11 = np.array(w2v_feats[11].detach()[0].cpu())
        w2v12 = np.array(w2v_feats[12].detach()[0].cpu())
        
#         egmaps_frame = signal.resample(egmaps, len(w2v0))
        
#         egmaps_phone = []
#         for i in range(0,len(egmaps)-3, 3):
#             egmaps_phone.append((egmaps[i]+egmaps[i+1]+egmaps[i+2])/3.0) #phoneme level   
#         egmaps_word=[]
#         for i in range(0, len(egmaps_phone)-5, 5):
#             egmaps_word.append((egmaps_phone[i]+egmaps_phone[i+1]+egmaps_phone[i+2]+egmaps_phone[i+3]+egmaps_phone[i+4])/5.0) #word level
        egmaps_utter = np.mean(egmaps, axis=0).reshape(1, -1)
#         print(egmaps_frame.shape)
#         print(w2v0.shape)
#         w2v0 = signal.resample(w2v0, len(egmaps_word))
#         w2v1 = signal.resample(w2v1, len(egmaps_word))
#         w2v2 = signal.resample(w2v2, len(egmaps_word))
#         w2v3 = signal.resample(w2v3, len(egmaps_word))
#         w2v4 = signal.resample(w2v4, len(egmaps_word))
#         w2v5 = signal.resample(w2v5, len(egmaps_word))
#         w2v6 = signal.resample(w2v6, len(egmaps_word))
#         w2v7 = signal.resample(w2v7, len(egmaps_word))
#         w2v8 = signal.resample(w2v8, len(egmaps_word))
#         w2v9 = signal.resample(w2v9, len(egmaps_word))
#         w2v10 = signal.resample(w2v10, len(egmaps_word))
#         w2v11 = signal.resample(w2v11, len(egmaps_word))
#         w2v12 = signal.resample(w2v12, len(egmaps_word))

#for utterance level
        w2v0 = np.mean(signal.resample(w2v0, 2), axis=0).reshape(1, -1)
        w2v1 = np.mean(signal.resample(w2v1, 2), axis=0).reshape(1, -1)
        w2v2 = np.mean(signal.resample(w2v2, 2), axis=0).reshape(1, -1)
        w2v3 = np.mean(signal.resample(w2v3, 2), axis=0).reshape(1, -1)
        w2v4 = np.mean(signal.resample(w2v4, 2), axis=0).reshape(1, -1)
        w2v5 = np.mean(signal.resample(w2v5, 2), axis=0).reshape(1, -1)
        w2v6 = np.mean(signal.resample(w2v6, 2), axis=0).reshape(1, -1)
        w2v7 = np.mean(signal.resample(w2v7, 2), axis=0).reshape(1, -1)
        w2v8 = np.mean(signal.resample(w2v8, 2), axis=0).reshape(1, -1)
        w2v9 = np.mean(signal.resample(w2v9, 2), axis=0).reshape(1, -1)
        w2v10 = np.mean(signal.resample(w2v10, 2), axis=0).reshape(1, -1)
        w2v11 = np.mean(signal.resample(w2v11, 2), axis=0).reshape(1, -1)
        w2v12 = np.mean(signal.resample(w2v12, 2), axis=0).reshape(1, -1)
        
#         print(w2v1.shape, egmaps_phone.shape)

        torch.cuda.empty_cache()
        
        if len(hct_utter) > 0:
            w2v0_feats = np.concatenate((w2v0_feats, w2v0), axis=0)
            w2v1_feats = np.concatenate((w2v1_feats, w2v1), axis=0)
            w2v2_feats = np.concatenate((w2v2_feats, w2v2), axis=0)
            w2v3_feats = np.concatenate((w2v3_feats, w2v3), axis=0)
            w2v4_feats = np.concatenate((w2v4_feats, w2v4), axis=0)
            w2v5_feats = np.concatenate((w2v5_feats, w2v5), axis=0)
            w2v6_feats = np.concatenate((w2v6_feats, w2v6), axis=0)
            w2v7_feats = np.concatenate((w2v7_feats, w2v7), axis=0)
            w2v8_feats = np.concatenate((w2v8_feats, w2v8), axis=0)
            w2v9_feats = np.concatenate((w2v9_feats, w2v9), axis=0)
            w2v10_feats = np.concatenate((w2v10_feats, w2v10), axis=0)
            w2v11_feats = np.concatenate((w2v11_feats, w2v11), axis=0)
            w2v12_feats = np.concatenate((w2v12_feats, w2v12), axis=0)
            hct_utter = np.concatenate((hct_utter, egmaps_utter), axis=0)
        else:
            w2v0_feats = w2v0
            w2v1_feats = w2v1
            w2v2_feats = w2v2
            w2v3_feats = w2v3
            w2v4_feats = w2v4
            w2v5_feats = w2v5
            w2v6_feats = w2v6
            w2v7_feats = w2v7
            w2v8_feats = w2v8
            w2v9_feats = w2v9
            w2v10_feats = w2v10 
            w2v11_feats = w2v11
            w2v12_feats = w2v12
            hct_utter = egmaps_utter
            
        n += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
corr_frame0, _, coefs0 = pwcca.compute_pwcca(hct_utter.T, w2v0_feats.T, epsilon=1e-10)
corr_frame1, _, coefs1 = pwcca.compute_pwcca(hct_utter.T, w2v1_feats.T, epsilon=1e-10)
corr_frame2, _, coefs2 = pwcca.compute_pwcca(hct_utter.T, w2v2_feats.T, epsilon=1e-10)
corr_frame3, _, coefs3 = pwcca.compute_pwcca(hct_utter.T, w2v3_feats.T, epsilon=1e-10)
corr_frame4, _, coefs4 = pwcca.compute_pwcca(hct_utter.T, w2v4_feats.T, epsilon=1e-10)
corr_frame5, _, coefs5 = pwcca.compute_pwcca(hct_utter.T, w2v5_feats.T, epsilon=1e-10)
corr_frame6, _, coefs6 = pwcca.compute_pwcca(hct_utter.T, w2v6_feats.T, epsilon=1e-10)
corr_frame7, _, coefs7 = pwcca.compute_pwcca(hct_utter.T, w2v7_feats.T, epsilon=1e-10)
corr_frame8, _, coefs8 = pwcca.compute_pwcca(hct_utter.T, w2v8_feats.T, epsilon=1e-10)
corr_frame9, _, coefs9 = pwcca.compute_pwcca(hct_utter.T, w2v9_feats.T, epsilon=1e-10)
corr_frame10, _, coefs10 = pwcca.compute_pwcca(hct_utter.T, w2v10_feats.T, epsilon=1e-10)
corr_frame11, _, coefs11 = pwcca.compute_pwcca(hct_utter.T, w2v11_feats.T, epsilon=1e-10)
corr_frame12, _, coefs12 = pwcca.compute_pwcca(hct_utter.T, w2v12_feats.T, epsilon=1e-10)

print('Layer 0')
# print(corr_frame0)
print(np.mean(coefs0))

print('Layer 1')
# print(corr_frame1)
print(np.mean(coefs1))

print('Layer 2')
# print(corr_frame2)
print(np.mean(coefs2))

print('Layer 3')
# print(corr_frame3)
print(np.mean(coefs3))

print('Layer 4')
# print(corr_frame4)
print(np.mean(coefs4))

print('Layer 5')
# print(corr_frame5)
print(np.mean(coefs5))

print('Layer 6')
# print(corr_frame6)
print(np.mean(coefs6))

print('Layer 7')
# print(corr_frame7)
print(np.mean(coefs7))

print('Layer 8')
# print(corr_frame8)
print(np.mean(coefs8))

print('Layer 9')
# print(corr_frame9)
print(np.mean(coefs9))

print('Layer 10')
# print(corr_frame10)
print(np.mean(coefs10))

print('Layer 11')
# print(corr_frame11)
print(np.mean(coefs11))

print('Layer 12')
# print(corr_frame12)
print(np.mean(coefs12))

Layer 0
0.8936124930426915
Layer 1
0.8992486017779552
Layer 2
0.8977664779007881
Layer 3
0.8980952635699664
Layer 4
0.889794280950146
Layer 5
0.8830500301561778
Layer 6
0.8792588679293658
Layer 7
0.8779646699565745
Layer 8
0.8765673800016721
Layer 9
0.8825377845949013
Layer 10
0.8896652321243824
Layer 11
0.8780853892595504
Layer 12
0.8797451041808251


In [ ]:
#PT

--frame level--
Layer 0
0.5841943796135864
Layer 1
0.6021493922045866
Layer 2
0.5982460915690141
Layer 3
0.5857582719880895
Layer 4
0.56658193411116
Layer 5
0.5445621839903267
Layer 6
0.5279630683104422
Layer 7
0.5200444966228587
Layer 8
0.5235861290659641
Layer 9
0.5479541783118378
Layer 10
0.5752668392087084
Layer 11
0.405608146595668
Layer 12
0.4811941764741984

--phoneme level--
Layer 0
0.5993155183309384
Layer 1
0.6201719894075368
Layer 2
0.6161960132019728
Layer 3
0.6038882775927162
Layer 4
0.5848705176524586
Layer 5
0.5627587318250595
Layer 6
0.5464921183101883
Layer 7
0.5387733871307925
Layer 8
0.5423727421807715
Layer 9
0.5663953648628862
Layer 10
0.5934620850022515
Layer 11
0.43253221493398086
Layer 12
0.5044028700686257

--word level--
Layer 0
0.652253153818508
Layer 1
0.6730757788810677
Layer 2
0.66855332157082
Layer 3
0.6562535025133178
Layer 4
0.637576584272603
Layer 5
0.6166318379336466
Layer 6
0.6000018620340711
Layer 7
0.5919748412321137
Layer 8
0.5939367442905545
Layer 9
0.6171276062982549
Layer 10
0.6412929872558709
Layer 11
0.5306894740328171
Layer 12
0.577477331490143

--utter level--
Layer 0
0.8936124930426915
Layer 1
0.8992486017779552
Layer 2
0.8977664779007881
Layer 3
0.8980952635699664
Layer 4
0.889794280950146
Layer 5
0.8830500301561778
Layer 6
0.8792588679293658
Layer 7
0.8779646699565745
Layer 8
0.8765673800016721
Layer 9
0.8825377845949013
Layer 10
0.8896652321243824
Layer 11
0.8780853892595504
Layer 12
0.8797451041808251

In [ ]:
#FT

--frame level--
Layer 0
0.5829933190839074
Layer 1
0.5912509982940299
Layer 2
0.5866393690367934
Layer 3
0.5737395201201761
Layer 4
0.5484999919370208
Layer 5
0.5247294468557037
Layer 6
0.5057026562408733
Layer 7
0.4917231310082758
Layer 8
0.4837699391545176
Layer 9
0.4822877026844705
Layer 10
0.4724109948926828
Layer 11
0.4390241362909809
Layer 12
0.42004541099173964

--phone level--
Layer 0
0.5984605151098696
Layer 1
0.6095964488052775
Layer 2
0.6047377931968285
Layer 3
0.5921317034515973
Layer 4
0.5669441906502485
Layer 5
0.5426040986920255
Layer 6
0.5234125554847253
Layer 7
0.5097031346437728
Layer 8
0.5016339215472102
Layer 9
0.49922728241151626
Layer 10
0.4879016217173191
Layer 11
0.4548163623099167
Layer 12
0.43607754009817845

--word level--
Layer 0
0.6531373948804858
Layer 1
0.6641670646503484
Layer 2
0.6592603407255813
Layer 3
0.6454952748703338
Layer 4
0.6217194607435653
Layer 5
0.597525917187226
Layer 6
0.5781117612164708
Layer 7
0.5650155365887165
Layer 8
0.5567880915140144
Layer 9
0.554882215329293
Layer 10
0.5466968069426211
Layer 11
0.5213010082249689
Layer 12
0.4873998030132867

--utter level--
Layer 0
0.8945177719445113
Layer 1
0.8966705565498758
Layer 2
0.895363613545673
Layer 3
0.8930714735761288
Layer 4
0.8842045688564548
Layer 5
0.8777519684448677
Layer 6
0.872988392590461
Layer 7
0.8678426058628285
Layer 8
0.8676767627814325
Layer 9
0.8670687413274275
Layer 10
0.8614647109327045
Layer 11
0.8541494017480558
Layer 12
0.8190397938108672

In [1]:
#phoneme level - frame level  PT model
print( 0.5993155183309384  -  0.58419437961358 )
print( 0.6201719894075368  -  0.60214939220458 )
print( 0.6161960132019728  -  0.59824609156901 )
print( 0.6038882775927162  -  0.58575827198808 )
print( 0.5848705176524586  -  0.56658193411116 )
print( 0.5627587318250595  -  0.54456218399032 )
print( 0.5464921183101883  -  0.52796306831044 )
print( 0.5387733871307925  -  0.52004449662285 )
print( 0.5423727421807715  -  0.52358612906596 )
print( 0.5663953648628862  -  0.54795417831183 )
print( 0.5934620850022515  -  0.57526683920870 )
print( 0.4325322149339808  -  0.40560814659566 )
print( 0.5044028700686257  -  0.48119417647419 )

0.015121138717358429
0.018022597202956847
0.01794992163296283
0.01813000560463629
0.018288583541298586
0.018196547834739563
0.01852904999974825
0.018728890507942464
0.018786613114811446
0.0184411865510562
0.018195245793551473
0.026924068338320783
0.023208693594435725


In [2]:
#word level - phoneme level  PT model
print( 0.65225315381850  -  0.5993155183309384 )
print( 0.67307577888106  -  0.6201719894075368 )
print( 0.66855332157082  -  0.6161960132019728 )
print( 0.65625350251331  -  0.6038882775927162 )
print( 0.63757658427260  -  0.5848705176524586 )
print( 0.61663183793364  -  0.5627587318250595 )
print( 0.60000186203407  -  0.5464921183101883 )
print( 0.59197484123211  -  0.5387733871307925 )
print( 0.59393674429055  -  0.5423727421807715 )
print( 0.61712760629825  -  0.5663953648628862 )
print( 0.64129298725587  -  0.5934620850022515 )
print( 0.53068947403281  -  0.4325322149339808 )
print( 0.57747733149014  -  0.5044028700686257 )

0.052937635487561585
0.052903789473523144
0.05235730836884722
0.05236522492059381
0.052706066620141434
0.053873106108580426
0.05350974372388173
0.05320145410131749
0.05156400210977852
0.05073224143536381
0.047830902253618524
0.09815725909882916
0.0730744614215143


In [3]:
#phoneme level - frame level  FT960 model

print( 0.5984605151098696  -  0.5829933190839074 )
print( 0.6095964488052775  -  0.5912509982940299 )
print( 0.6047377931968285  -  0.5866393690367934 )
print( 0.5921317034515973  -  0.5737395201201761 )
print( 0.5669441906502485  -  0.5484999919370208 )
print( 0.5426040986920255  -  0.5247294468557037 )
print( 0.5234125554847253  -  0.5057026562408733 )
print( 0.5097031346437728  -  0.4917231310082758 )
print( 0.5016339215472102  -  0.4837699391545176 )
print( 0.4992272824115162  -  0.4822877026844705 )
print( 0.4879016217173191  -  0.4724109948926828 )
print( 0.4548163623099167  -  0.4390241362909809 )
print( 0.4360775400981784  -  0.4200454109917396 )

0.015467196025962249
0.018345450511247585
0.018098424160035154
0.01839218333142112
0.018444198713227666
0.017874651836321864
0.017709899243852023
0.01798000363549701
0.0178639823926926
0.016939579727045684
0.015490626824636311
0.015792226018935762
0.01603212910643881


In [4]:
#word level - phoneme level  FT960 model

print( 0.653137394880485  -  0.5984605151098696 )
print( 0.664167064650348  -  0.6095964488052775 )
print( 0.659260340725581  -  0.6047377931968285 )
print( 0.645495274870333  -  0.5921317034515973 )
print( 0.621719460743565  -  0.5669441906502485 )
print( 0.597525917187226  -  0.5426040986920255 )
print( 0.578111761216470  -  0.5234125554847253 )
print( 0.565015536588716  -  0.5097031346437728 )
print( 0.556788091514014  -  0.5016339215472102 )
print( 0.554882215329293  -  0.4992272824115162 )
print( 0.546696806942621  -  0.4879016217173191 )
print( 0.521301008224968  -  0.4548163623099167 )
print( 0.487399803013286  -  0.4360775400981784 )

0.054676879770615305
0.0545706158450705
0.05452254752875252
0.05336357141873571
0.05477527009331651
0.0549218184952005
0.05469920573174469
0.055312401944943135
0.05515416996680378
0.055654932917776845
0.058795185225301916
0.06648464591505138
0.05132226291510761
